<a href="https://colab.research.google.com/github/ankit-rathi/Quantvesting_v2/blob/main/myStocks_Portfolio_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pyxirr
!pip install ta
!pip install yfinance==0.2.59

  Installing build dependencies ... -

 \

 done


  Getting requirements to build wheel ... -

 done


...

 done


 done
  Created wheel for ta: filename=ta-0.11.0-py3-none-any.whl size=29482 sha256=881cf3ffedcef21c9461f4902181f07ef944931a00a670dbccdbbe323881525c
  Stored in directory: /home/runner/.cache/pip/wheels/e3/3a/ee/4955a26c90a4b7deb6d725dc8ec7b8604a7aef44e43a2e8af7
Successfully built ta


  Attempting uninstall: protobuf
    Found existing installation: protobuf 6.32.1
    Uninstalling protobuf-6.32.1:
      Successfully uninstalled protobuf-6.32.1


  Attempting uninstall: yfinance
    Found existing installation: yfinance 0.2.66
    Uninstalling yfinance-0.2.66:
      Successfully uninstalled yfinance-0.2.66
   ━━━━━━━━━━━━━━━━━━━━╺━━━━━━━━━━━━━━━━━━━ 1/2 [yfinance]

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2/2 [yfinance]


In [2]:
# import libraries

import numpy as np
import pandas as pd
import datetime
import warnings
warnings.filterwarnings('ignore')
import yfinance as yfin
import ta
import matplotlib.pyplot as plt
import requests

Matplotlib is building the font cache; this may take a moment.


In [3]:
# notebook start time
import time
import datetime
import pytz

start_time = time.time()

# function to print date timestamp
def print_date_time():
  tz_NY = pytz.timezone('Asia/Kolkata')
  datetime_NY = datetime.datetime.now(tz_NY)
  print("Run date time (IST):", datetime_NY.strftime("%Y-%m-%d %H:%M:%S"))

In [4]:
# function to get booked and reserved amount
def get_amt():
  # fetch the JSON data from the URL
  url = "https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioAmts.json"
  response = requests.get(url)
  pf_amts = response.json()  # parse the JSON data

  # extract values from the JSON
  py_booked_amt_dm = pf_amts["py_booked_amt_dm"]
  py_booked_amt_sv = pf_amts["py_booked_amt_sv"]
  cy_booked_amt_dm = pf_amts["cy_booked_amt_dm"]
  cy_booked_amt_sv = pf_amts["cy_booked_amt_sv"]
  reserve_amt_dm = pf_amts["reserve_amt_dm"]
  reserve_amt_sv = pf_amts["reserve_amt_sv"]

  # perform calculations
  py_booked_amt = py_booked_amt_dm + py_booked_amt_sv
  cy_booked_amt = cy_booked_amt_dm + cy_booked_amt_sv
  reserve_amt = reserve_amt_dm + reserve_amt_sv
  total_booked_amt = py_booked_amt + cy_booked_amt

  return total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt

gen_output = 0

In [5]:
# function to format the amount
def format_amt(number):
    abs_number = abs(number)

    if abs_number >= 1_00_00_000:  # Crores
        formatted_amt = f"{number / 1_00_00_000:.2f} C"
    elif abs_number >= 1_00_000:  # Lakhs
        formatted_amt = f"{number / 1_00_000:.2f} L"
    elif abs_number >= 1_000:  # Thousands
        formatted_amt = f"{number / 1_000:.2f} K"
    else:
        formatted_amt = f"{number:.2f}"

    return formatted_amt

# function to set start and end date
def get_start_end_date():
  start_date = (datetime.date.today() + datetime.timedelta(days=-365)).strftime('%Y-%m-%d')
  end_date = (datetime.date.today() + datetime.timedelta(days=1)).strftime('%Y-%m-%d')
  return start_date, end_date

# function to fetch my portfolio csv
def get_mypfs_df():
  mypfs_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioStocks.csv')
  return mypfs_df

# function to fetch my prospects csv
def get_mypps_df():
  mypps_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myProspectsScrips.csv')
  return mypps_df

# function to fetch screener data
def get_myscreen_df():
  myscreen_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myScreenerDB.csv')
  return myscreen_df

# function to fetch momentum data
def get_myinvmt_df():
  myinvmt_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myInvestments.csv')
  return myinvmt_df

# function to get the stock ids
def get_stock_ids(df_pf):
  stock_n100 = df_pf['Symbol'].unique()

  exclude = ['CADILAHC','MMTC', 'MASFIN']

  stock_ids = df_pf[~df_pf['Symbol'].isin(exclude) ]['Symbol'].unique()

  #mypf = mypf[mypf.Forecast.notnull()]
  #stock_ids = mypf['Symbol'].unique()

  stock_ids.sort()
  return stock_ids

# get features from screener data
def get_screener_features():
  myscreen_df = get_myscreen_df()
  cols = ['Symbol', 'EPS', 'MedPE', 'ROCE%', 'ROE%', 'CapType']
  return myscreen_df[cols]

# get relative strength
def get_relative_strength(stock_list):

    # Dictionary to store stock tickers and their corresponding percentage price change
    stock_changes = {}

    # Loop through each stock and fetch its price data
    for stock in stock_list:
        try:
            # Download the stock data for the given date range
            stock_data = yfin.Ticker(stock + '.NS').history(period='1mo', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

            # Calculate the percentage change for the stock
            if len(stock_data) > 0:
                start_price = stock_data['Close'].iloc[0]
                end_price = stock_data['Close'].iloc[-1]
                percent_change = round(((end_price - start_price) / start_price) * 100, 2)
                stock_changes[stock] = percent_change
            else:
                stock_changes[stock] = np.nan  # If no data is available, set to NaN

        except Exception as e:
            print(f"Error fetching data for {stock}: {e}")
            stock_changes[stock] = np.nan

    # Create a DataFrame with stock tickers and their percentage changes
    df = pd.DataFrame(list(stock_changes.items()), columns=['Symbol', 'Percent_Change'])

    # Drop any stocks with missing data (NaN values)
    df = df.dropna()

    # Calculate the percentile rank based on percentage change
    df['RSP'] = round(df['Percent_Change'].rank(pct=True) * 100, 2)
    cols = ['Symbol', 'RSP']
    df = df[cols]
    # Sort by percentile rank (optional)
    df = df.sort_values(by='RSP', ascending=False).reset_index(drop=True)

    return df


In [6]:
# function to get stock technicals
def stock_prec_dev(stock_symbol):
    #stock_symbol = 'ULTRACEMCO.NS'
    short_window = 20
    mid_window = 50
    long_window = 200
    moving_avg = 'SMA'

    start = datetime.datetime(*map(int, start_date.split('-')))
    end = datetime.datetime(*map(int, end_date.split('-')))

    stock_df = yfin.Ticker(stock_symbol + '.NS').history(period='max', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

    stock_df['Max'] = round(max(stock_df['Close']),2)
    stock_df = stock_df[(stock_df.index <= end_date) & (stock_df.index >= start_date)]
    stock_df['200_SMA'] = round(stock_df['Close'].rolling(window = 200, min_periods = 1).mean(),0)
    stock_df['Dev%_200'] = round((stock_df['Close'] - stock_df['200_SMA'])*100/stock_df['200_SMA'],2)
    stock_df.dropna(axis = 0, inplace = True) # remove any null rows

    stock_df['20_SMA'] = stock_df['Close'].rolling(window=20).mean()
    stock_df['50_SMA'] = stock_df['Close'].rolling(window=50).mean()
    stock_df['Symbol'] = stock_symbol

    stock_df['Close'] = round(stock_df['Close'],2)
    stock_df['Min'] = round(min(stock_df['Close']),2)
    stock_df['RSI_14'] = ta.momentum.RSIIndicator(close=stock_df['Close'], window=14).rsi()
    stock_df['RSI_14'] = round(stock_df['RSI_14'],0)
    stock_df['Prev_Close'] = stock_df['Close'].shift(1)
    stock_df.drop(['Open', 'Low', 'High', 'Volume'], axis=1, inplace=True)
    stock_df = stock_df.tail(1)

    max_SMA = max(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    min_SMA = min(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    ABS_Spread = max_SMA - min_SMA
    stock_df['Spread%'] = round((ABS_Spread / stock_df['200_SMA'].item()) * 100,2)

    return stock_df

# function to compute stock attributes
def get_common_features(stock_ids, df_mypf):

  df_prec_dev = pd.DataFrame()
  df_tmp = get_screener_features()
  df_rs = get_relative_strength(stock_ids)

  for stock_id in stock_ids:
      tmp = stock_prec_dev(stock_id)
      tmp = tmp.reset_index()
      df_prec_dev = pd.concat([df_prec_dev, tmp], ignore_index = True)
  df_prec_dev = pd.merge(df_prec_dev, df_mypf, on= 'Symbol')
  df_prec_dev = pd.merge(df_prec_dev, df_tmp, on= 'Symbol', how='left')
  df_prec_dev = pd.merge(df_prec_dev, df_rs, on= 'Symbol', how='left')
  #print(df_prec_dev.columns)
  df_prec_dev['Curr_PE'] = round(df_prec_dev['Close']/df_prec_dev['EPS'],1)
  df_prec_dev['Dev%_PE'] = round((df_prec_dev['Curr_PE'] - df_prec_dev['MedPE'])*100/df_prec_dev['MedPE'],2)
  df_prec_dev['Conviction'] = df_prec_dev['Conviction'] + '-' + df_prec_dev['CapType']
  return df_prec_dev

# function to arrange stock features
def arrange_features(df_stocks, common_cols, diff_cols):
  df_stocks_common = df_stocks[common_cols].drop_duplicates()
  df_stocks_diff = df_stocks[diff_cols]
  df_stocks_diff['Investment'] = df_stocks_diff['AvgCost'] * df_stocks_diff['Shares']
  df_stocks_diff = df_stocks_diff.groupby(['Symbol'])[['Shares', 'Investment']].aggregate(['sum']).reset_index()
  df_stocks_diff.columns = ['Symbol', 'Shares', 'Investment']
  df_stocks_diff['AvgCost'] = round(df_stocks_diff['Investment']/df_stocks_diff['Shares'],2)
  df_stocks = pd.merge(df_stocks_diff, df_stocks_common, on='Symbol')
  return df_stocks

# plot fact distribution across dimension
def plot_pie_chart(df, dimension, fact):
  # grouping the data by category and calculating the sum of fact for each type
  grouped_data = df.groupby(dimension)[fact].sum()

  # sorting the grouped data in descending order
  grouped_data = grouped_data.sort_values(ascending=False)

  # creating a pie chart
  grouped_data.plot.pie(autopct='%1.1f%%', startangle=90, figsize=(6, 6))

  # adding a title and displaying the plot
  plt.title(f'{dimension} {fact} Distribution')
  plt.ylabel('')  # To hide the y-label
  plt.show()

In [7]:
# function to get portfolio features

def get_portfolio_features(df_common_features):

  print_date_time()
  print('-------------------')

  #df_common_features["Target"] = df_common_features["Target"].fillna(df_common_features["Max"])
  df_common_features['NTT'] = np.where(df_common_features['Strategy']== 'NTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['LTT'] = np.where(df_common_features['Strategy']== 'BTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['BOL'] = df_common_features['Min']

  tmp_df = df_common_features[df_common_features['Symbol'].isin(mypf_df[mypf_df['InPortfolio'] != 'NA'].Symbol.values)]
  print('qualified stocks: '+str(len(tmp_df['Symbol'].unique())))
  tmp_df1 = tmp_df[tmp_df['LatestQtr'] == 1]
  print('with latest results: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df1 = tmp_df1[tmp_df1['StarStock'] == 1]
  print('still star stocks: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df['Investment'] = tmp_df['AvgCost'] * tmp_df['Shares']
  tmp_df['Current'] = round(tmp_df['Close'] * tmp_df['Shares'],0)
  tmp_df['Previous'] = tmp_df['Prev_Close'] * tmp_df['Shares']
  tmp_df['EstimatedST'] = tmp_df['NTT'] * tmp_df['Shares']
  tmp_df['EstimatedLT'] = tmp_df['LTT'] * tmp_df['Shares']
  tmp_df['Current P/L'] = round((tmp_df['Current'] - tmp_df['Investment']),0)
  tmp_df['Today P/L%'] = round((tmp_df['Current'] - tmp_df['Previous'])*100/tmp_df['Previous'],2)
  tmp_df['Current P/L%'] = round((tmp_df['Current'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedST P/L%'] = round((tmp_df['EstimatedST'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedLT P/L%'] = round((tmp_df['EstimatedLT'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['NTT%'] = round((tmp_df['NTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['LTT%'] = round((tmp_df['LTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['Gained%'] = round((tmp_df['Close'] - tmp_df['BOL'])*100/tmp_df['BOL'],2)
  investment = round(sum(tmp_df['AvgCost']*tmp_df['Shares']),0)
  current = round(sum(tmp_df['Close']*tmp_df['Shares']),0)
  tmp_df['InitAlloc%'] = round(tmp_df['Investment']*100/investment,2)
  tmp_df['CurrAlloc%'] = round(tmp_df['Current']*100/current,2)
  tmp_df['FTT'] = tmp_df['LTT']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT'] = tmp_df['NTT']
  tmp_df['FTT%'] = tmp_df['LTT%']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT%'] = tmp_df['NTT%']
  tmp_df['FTT Amt'] = round(tmp_df['FTT%'] * tmp_df['Current']/100,0)
  tmp_df['OTT%'] = round((tmp_df['FTT'] - tmp_df['AvgCost'])*100/tmp_df['AvgCost'],2)
  tmp_df['RRR Ind'] = round(tmp_df['Current P/L']/tmp_df['FTT Amt'],2)
  tmp_df['Risk Ind'] = round(tmp_df['Current P/L%']*tmp_df['CurrAlloc%'],0)

  return tmp_df

# function to print portfolio features
def print_portfolio_stats(df_portfolio_features, myinvmt_df):
  from pyxirr import xirr

  total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt = get_amt()

  dates = myinvmt_df['Date'].values
  dates = np.append(dates, datetime.date.today().strftime('%d-%b-%y'))
  investment = myinvmt_df['Investment'].values
  dates= pd.to_datetime(dates)

  current = round(sum(df_portfolio_features['Close']*df_portfolio_features['Shares']),0) + reserve_amt
  investment_xirr = np.append(investment, current)
  cagr = round(xirr(pd.DataFrame({"dates": dates, "amounts": investment_xirr}))*100,2)

  investment = -sum(investment)
  invested = round(sum(df_portfolio_features['AvgCost']*df_portfolio_features['Shares']),0) + reserve_amt
  previous = round(sum(df_portfolio_features['Prev_Close']*df_portfolio_features['Shares']),0) + reserve_amt
  cy_invested = investment + py_booked_amt

  today_pnl_amount = current-previous
  today_pnl_percentage = round((current-previous)*100/previous,2)

  curr_pnl_amount = current-invested
  curr_pnl_percentage = round((curr_pnl_amount)*100/(cy_invested),2)

  cy_pnl_amount = cy_booked_amt + curr_pnl_amount
  cy_pnl_percentage = round((cy_pnl_amount)*100/cy_invested,2)

  overall_pnl_amount = total_booked_amt + curr_pnl_amount
  overall_pnl_percentage = round((overall_pnl_amount)*100/investment,2)

  estimate_st = round(sum(df_portfolio_features['FTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_st_pnl_amount = estimate_st-current
  est_st_pnl_percentage = round((est_st_pnl_amount)*100/current,2)

  estimate_lt = round(sum(df_portfolio_features['LTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_lt_pnl_amount = estimate_lt-current
  est_lt_pnl_percentage = round((est_lt_pnl_amount)*100/current,2)

  total_profit = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Investment']),0)
  total_loss = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Investment']),0)

  cy_booked_percentage = round((cy_booked_amt)*100/current,2)
  py_booked_percentage = round((py_booked_amt)*100/investment,2)
  total_booked_percentage = round((total_booked_amt)*100/investment,2)

  print('-------------------')
  print('Initial Investment: ', format_amt(investment))
  print('CY Investment: ', format_amt(cy_invested))
  print('Reserve: ', format_amt(reserve_amt))
  print('Current: ',  format_amt(current))
  print('-------------------')
  print('Today PnL: '+ '{} ({}%)'.format(format_amt(today_pnl_amount), today_pnl_percentage))
  print('Current PnL: '+ '{} ({}%)'.format(format_amt(curr_pnl_amount), curr_pnl_percentage))
  print('CY Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(cy_pnl_amount), cy_pnl_percentage))
  print('-------------------')
  print('Total profit: ', format_amt(total_profit))
  print('Total loss: ', format_amt(total_loss))
  print('-------------------')
  print('Total Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(overall_pnl_amount), overall_pnl_percentage))
  print('Total Booked PnL: '+ '{} ({}%)'.format(format_amt(total_booked_amt), total_booked_percentage))
  print('Curr Year Booked PnL: '+ '{} ({}%)'.format(format_amt(cy_booked_amt), cy_booked_percentage))
  print('Prev Year Booked PnL: '+ '{} ({}%)'.format(format_amt(py_booked_amt), py_booked_percentage))
  print('===================')
  print('Est FTT: ',  format_amt(estimate_st))
  print('Est FTT PnL: '+ '{} ({}%)'.format(format_amt(est_st_pnl_amount), est_st_pnl_percentage))

  print('===================')
  print('Deployed: ', format_amt(investment))

  print('Current: ', format_amt(current))

  print('CAGR/XIRR %: '+'{}%'.format(cagr))

In [8]:
# get start and end date
start_date, end_date = get_start_end_date()

# get portfolio and prospects data
mypfs_df = get_mypfs_df()
mypps_df = get_mypps_df()
myinvmt_df = get_myinvmt_df()

# merge above datasets
mypf_df = pd.merge(mypfs_df, mypps_df, on="Symbol")

# seggregate dm and sv portfolio
dm_pf = mypf_df[mypf_df['InPortfolio'] == 'DM']
sv_pf = mypf_df[mypf_df['InPortfolio'] == 'SV']

# build portfolio stock dataframe
dm_stocks = get_stock_ids(dm_pf)
sv_stocks = get_stock_ids(sv_pf)
df_stocks = pd.concat([dm_pf,sv_pf], ignore_index = True)

# arrange common and diff stock features
common_cols = ['Symbol', 'Target', 'Criteria', 'Strategy', 'CumlRnk', 'LatestQtr', 'StarStock', 'Conviction', 'Category', 'InFolio', 'XIRR', 'MBQ']
diff_cols = ['Symbol', 'AvgCost', 'Shares']
df_stocks = arrange_features(df_stocks, common_cols, diff_cols)

# get common features
stock_ids = df_stocks['Symbol'].values
df_common_features = get_common_features(stock_ids, df_stocks)

df_common_features.reset_index(drop=True, inplace=True)
df_common_features.drop(['Date'], axis=1, inplace=True)
# get and print portfolio features
df_portfolio_features = get_portfolio_features(df_common_features)

print_portfolio_stats(df_portfolio_features, myinvmt_df)

df = df_portfolio_features
#plot_pie_chart(df, 'CapType', 'Current')
list_ox40n = list(df[df['MBQ'].str.contains('OX40N', na=False)]['Symbol'].values)

Run date time (IST): 2025-09-26 13:44:37
-------------------
qualified stocks: 87
with latest results: 27
still star stocks: 16
-------------------
Initial Investment:  1.22 C
CY Investment:  1.49 C
Reserve:  4.38 K
Current:  1.36 C
-------------------
Today PnL: -2.05 L (-1.48%)
Current PnL: -23.11 L (-15.55%)
CY Booked + Current PnL: -11.67 L (-7.85%)
-------------------
Total profit:  80.20 K
Total loss:  -23.91 L
-------------------
Total Booked + Current PnL: 15.31 L (12.58%)
Total Booked PnL: 38.42 L (31.57%)
Curr Year Booked PnL: 11.44 L (8.38%)
Prev Year Booked PnL: 26.98 L (22.17%)
Est FTT:  2.25 C
Est FTT PnL: 88.98 L (65.19%)
Deployed:  1.22 C
Current:  1.36 C
CAGR/XIRR %: 6.95%


In [9]:
# top 5 near their targets
cols = ['Symbol', 'FTT', 'Dev%_PE', 'RSI_14','Conviction', 'Spread%', 'Current', 'Current P/L', 'FTT Amt', 'Today P/L%', 'Current P/L%', 'FTT%', 'OTT%', 'CumlRnk', 'RRR Ind', 'CurrAlloc%', 'Gained%', 'Criteria', 'Strategy', 'Category']
df_tmp = df_portfolio_features[~df_portfolio_features['Symbol'].isin(['ENRIN','BLUSPRING','DIGITIDE'])]
df_tmp = df_tmp.sort_values(by = 'FTT Amt', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
77,TTKPRESTIG,770.00,93.38,36.0,M-SC,2.75,83355.0,-17422.0,17513.0,-2.59,-17.29,21.01,0.09,245.0,-0.99,0.61,9.51,OX40N,NTT,DURABLES
14,BLUESTARCO,2080.00,9.17,46.0,H-MC,3.81,188460.0,23790.0,19543.0,-2.22,14.45,10.37,26.31,89.0,1.22,1.38,23.58,X40N,NTT,AC
42,ITC,452.00,-38.77,45.0,H-LC,2.15,197996.0,-2142.0,23027.0,1.20,-1.07,11.63,10.44,4.0,-0.09,1.45,4.43,X40,NTT,FMCG
50,NATIONALUM,244.55,-46.98,47.0,H-MC,10.89,106445.0,5941.0,23162.0,-2.32,5.91,21.76,28.96,79.0,0.26,0.78,41.89,MH,ATH,METALS
57,RELIANCE,1533.00,-14.04,45.0,H-LC,4.07,215218.0,4852.0,23932.0,0.52,2.31,11.12,13.68,37.0,0.20,1.58,19.21,XY25,NTT,REFINERIES


In [10]:
# top 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
73,TATAMOTORS,1065.0,-22.82,42.0,M-LC,2.34,244571.0,-31952.0,142022.0,1.42,-11.55,58.07,39.81,54.0,-0.22,1.79,17.21,XY24,NTT,AUTO
42,ITC,452.0,-38.77,45.0,H-LC,2.15,197996.0,-2142.0,23027.0,1.20,-1.07,11.63,10.44,4.0,-0.09,1.45,4.43,X40,NTT,FMCG
18,CERA,9475.0,-22.20,39.0,H-SC,1.82,142014.0,-33889.0,75906.0,0.76,-19.27,53.45,23.89,149.0,-0.45,1.04,22.65,OX40N,NTT,CERAMICS
57,RELIANCE,1533.0,-14.04,45.0,H-LC,4.07,215218.0,4852.0,23932.0,0.52,2.31,11.12,13.68,37.0,0.20,1.58,19.21,XY25,NTT,REFINERIES
45,KANSAINER,340.0,-67.79,52.0,H-SC,1.43,221715.0,-47952.0,84296.0,0.45,-17.78,38.02,13.47,138.0,-0.57,1.63,12.14,XY24,NTT,PAINTS


In [11]:
# bottom 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
46,KPIGREEN,731.05,-4.80,29.0,H-SC,8.74,113576.0,-11721.0,70644.0,-4.22,-9.35,62.20,47.03,141.0,-0.17,0.83,42.21,MH,ATH,POWER
21,COFFEEDAY,80.00,-47.69,41.0,L-SC,41.27,77952.0,-35597.0,75645.0,-4.18,-31.35,97.04,35.27,268.0,-0.47,0.57,90.79,XR,NTT,HOTELS
60,SAIL,228.00,43.54,53.0,M-MC,11.13,226684.0,1722.0,165933.0,-3.57,0.77,73.20,74.53,192.0,0.01,1.66,33.27,XY24,BTT,STEEL
32,HINDZINC,730.22,20.20,52.0,M-LC,2.77,195859.0,-9217.0,121785.0,-3.53,-4.49,62.18,54.89,52.0,-0.08,1.44,18.52,X5K,ATH,METALS
39,INDUSINDBK,1800.00,232.63,35.0,L-MC,12.09,40838.0,-36368.0,61763.0,-3.28,-47.11,151.24,32.89,258.0,-0.59,0.30,12.46,XR,NTT,BANKS


In [12]:
# top 5 to exit based on CumlRnk
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > -1) & (df_tmp['Current P/L%'] < 1)].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
60,SAIL,228.00,43.54,53.0,M-MC,11.13,226684.0,1722.0,165933.0,-3.57,0.77,73.20,74.53,192.0,0.01,1.66,33.27,XY24,BTT,STEEL
11,BANKINDIA,190.00,-30.43,52.0,H-MC,8.18,180561.0,753.0,111659.0,-1.97,0.42,61.84,62.52,88.0,0.01,1.32,34.65,XR,NTT,BANKS
52,PGHH,17973.08,-30.28,51.0,H-MC,3.40,201945.0,1125.0,67652.0,-2.34,0.56,33.50,34.25,80.0,0.02,1.48,5.79,X40,ATH,FMCG


In [13]:
# OX40N stocks to exit
df_tmp = df_portfolio_features[(df_portfolio_features['Symbol'].isin(list_ox40n))].sort_values(by = 'Current P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
78,UJJIVANSFB,60.0,90.20,47.0,H-SC,10.14,120069.0,-22410.0,41928.0,-1.11,-15.73,34.92,13.70,163.0,-0.53,0.88,41.71,OX40N,NTT,BANKS
77,TTKPRESTIG,770.0,93.38,36.0,M-SC,2.75,83355.0,-17422.0,17513.0,-2.59,-17.29,21.01,0.09,245.0,-0.99,0.61,9.51,OX40N,NTT,DURABLES
45,KANSAINER,340.0,-67.79,52.0,H-SC,1.43,221715.0,-47952.0,84296.0,0.45,-17.78,38.02,13.47,138.0,-0.57,1.63,12.14,XY24,NTT,PAINTS
18,CERA,9475.0,-22.20,39.0,H-SC,1.82,142014.0,-33889.0,75906.0,0.76,-19.27,53.45,23.89,149.0,-0.45,1.04,22.65,OX40N,NTT,CERAMICS
66,SIS,528.0,2081.35,40.0,H-SC,4.23,88026.0,-23006.0,46610.0,-0.26,-20.72,52.95,21.26,156.0,-0.49,0.65,18.93,OX40N,NTT,MISC


In [14]:
# top 5 to book for rotation
df_tmp = df_portfolio_features[(df_portfolio_features['Current P/L%'] > 20) ].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category


In [15]:
# top 5 to monitor for exit
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > 0) ].sort_values(by = 'Dev%_PE', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
60,SAIL,228.00,43.54,53.0,M-MC,11.13,226684.0,1722.0,165933.0,-3.57,0.77,73.20,74.53,192.0,0.01,1.66,33.27,XY24,BTT,STEEL
14,BLUESTARCO,2080.00,9.17,46.0,H-MC,3.81,188460.0,23790.0,19543.0,-2.22,14.45,10.37,26.31,89.0,1.22,1.38,23.58,X40N,NTT,AC
84,VOLTAS,1530.00,-3.46,40.0,H-MC,3.03,202440.0,10698.0,27066.0,-1.66,5.58,13.37,19.69,99.0,0.40,1.48,12.76,XY25,NTT,AC
17,CAMS,5211.76,-6.05,44.0,H-SC,1.77,106904.0,4900.0,39020.0,-1.30,4.80,36.50,43.06,122.0,0.13,0.78,22.74,X40N,ATH,MISC
79,UNIONBANK,163.00,-14.73,50.0,M-LC,6.81,153290.0,12450.0,32038.0,-2.85,8.84,20.90,31.59,66.0,0.39,1.12,37.67,XY24,NTT,BANKS


In [16]:
# top 5 to book for rotation from weak conviction
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[~df_tmp['Criteria'].isin(['XY25','XY24','X40', 'X40N'])].sort_values(by = 'RRR Ind', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
50,NATIONALUM,244.55,-46.98,47.0,H-MC,10.89,106445.0,5941.0,23162.0,-2.32,5.91,21.76,28.96,79.0,0.26,0.78,41.89,MH,ATH,METALS
85,WHIRLPOOL,2270.00,-45.08,31.0,M-SC,1.53,92865.0,1366.0,77384.0,-0.86,1.49,83.33,86.07,223.0,0.02,0.68,34.86,XR,NTT,DURABLES
11,BANKINDIA,190.00,-30.43,52.0,H-MC,8.18,180561.0,753.0,111659.0,-1.97,0.42,61.84,62.52,88.0,0.01,1.32,34.65,XR,NTT,BANKS
37,INDIAMART,4850.92,-55.20,25.0,H-SC,10.23,125228.0,1892.0,131865.0,-0.46,1.53,105.30,108.45,119.0,0.01,0.92,24.23,AR,ATH,MISC
86,WIPRO,420.00,-16.52,33.0,M-LC,5.64,146946.0,-3999.0,113457.0,-2.06,-2.65,77.21,72.51,53.0,-0.04,1.08,3.21,XR,NTT,IT


In [17]:
# top 5 to accumulate based on lowest RSI_14
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'RSI_14', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
5,ASIANPAINT,3465.66,-18.03,28.0,H-LC,7.02,206870.0,-44898.0,98098.0,-2.21,-17.83,47.42,21.13,27.0,-0.46,1.52,11.27,X40,ATH,PAINTS
44,JIOFIN,387.00,-6.26,28.0,H-LC,12.95,254732.0,-12633.0,78075.0,-1.79,-4.73,30.65,24.48,48.0,-0.16,1.87,47.65,XY24,BTT,FINANCE
74,TCS,4476.75,-31.58,29.0,H-LC,12.08,272985.0,-72971.0,147821.0,-1.80,-21.09,54.15,21.64,1.0,-0.49,2.00,0.00,X40,ATH,IT
83,VBL,671.64,-20.64,30.0,H-LC,8.19,285633.0,-30209.0,142874.0,-0.92,-9.56,50.02,35.67,5.0,-0.21,2.09,2.96,X40N,ATH,FMCG
30,HCLTECH,1943.91,-4.09,34.0,H-LC,9.94,219219.0,-22705.0,85978.0,-1.93,-9.39,39.22,26.15,8.0,-0.26,1.61,3.59,X40,ATH,IT


In [18]:
# top 5 to accumulate based on Spread and CumlRnk
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['CumlRnk'] < 100)].sort_values(by = 'Spread%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
48,LTIM,7230.2,-7.51,40.0,H-LC,1.68,229612.0,-16539.0,95748.0,-2.14,-6.72,41.70,32.18,16.0,-0.17,1.68,26.90,X200,ATH,IT
42,ITC,452.0,-38.77,45.0,H-LC,2.15,197996.0,-2142.0,23027.0,1.20,-1.07,11.63,10.44,4.0,-0.09,1.45,4.43,X40,NTT,FMCG
65,SIEMENS,4671.5,-3.90,40.0,H-LC,2.63,154575.0,-31520.0,79003.0,-2.05,-16.94,51.11,25.51,15.0,-0.40,1.13,13.91,AR,ATH,ELECTRICAL
57,RELIANCE,1533.0,-14.04,45.0,H-LC,4.07,215218.0,4852.0,23932.0,0.52,2.31,11.12,13.68,37.0,0.20,1.58,19.21,XY25,NTT,REFINERIES
51,NESTLEIND,1377.0,-11.41,43.0,H-LC,4.52,272493.0,7067.0,49730.0,-0.77,2.66,18.25,21.40,11.0,0.14,2.00,9.94,XY25,NTT,FMCG


In [19]:
# top 5 to accumulate based on lowest Gained%
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'Gained%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
74,TCS,4476.75,-31.58,29.0,H-LC,12.08,272985.0,-72971.0,147821.0,-1.80,-21.09,54.15,21.64,1.0,-0.49,2.00,0.00,X40,ATH,IT
80,UNITDSPR,1644.00,-14.66,43.0,H-LC,7.17,223806.0,-10338.0,58951.0,-1.24,-4.42,26.34,20.77,86.0,-0.18,1.64,2.20,X40N,NTT,BREWERIES
83,VBL,671.64,-20.64,30.0,H-LC,8.19,285633.0,-30209.0,142874.0,-0.92,-9.56,50.02,35.67,5.0,-0.21,2.09,2.96,X40N,ATH,FMCG
30,HCLTECH,1943.91,-4.09,34.0,H-LC,9.94,219219.0,-22705.0,85978.0,-1.93,-9.39,39.22,26.15,8.0,-0.26,1.61,3.59,X40,ATH,IT
42,ITC,452.00,-38.77,45.0,H-LC,2.15,197996.0,-2142.0,23027.0,1.20,-1.07,11.63,10.44,4.0,-0.09,1.45,4.43,X40,NTT,FMCG


In [20]:
# top 5 to accumulate based on lowest CurrAlloc%
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
65,SIEMENS,4671.50,-3.90,40.0,H-LC,2.63,154575.0,-31520.0,79003.0,-2.05,-16.94,51.11,25.51,15.0,-0.40,1.13,13.91,AR,ATH,ELECTRICAL
42,ITC,452.00,-38.77,45.0,H-LC,2.15,197996.0,-2142.0,23027.0,1.20,-1.07,11.63,10.44,4.0,-0.09,1.45,4.43,X40,NTT,FMCG
19,CIPLA,1795.00,-23.21,36.0,H-LC,4.55,205240.0,740.0,40679.0,-0.77,0.36,19.82,20.25,10.0,0.02,1.50,9.13,X40N,BTT,PHARMA
5,ASIANPAINT,3465.66,-18.03,28.0,H-LC,7.02,206870.0,-44898.0,98098.0,-2.21,-17.83,47.42,21.13,27.0,-0.46,1.52,11.27,X40,ATH,PAINTS
57,RELIANCE,1533.00,-14.04,45.0,H-LC,4.07,215218.0,4852.0,23932.0,0.52,2.31,11.12,13.68,37.0,0.20,1.58,19.21,XY25,NTT,REFINERIES


In [21]:
# top 5 to accumulate based on CumlRnk
df_tmp = df_portfolio_features.sort_values(by = 'CumlRnk', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
74,TCS,4476.75,-31.58,29.0,H-LC,12.08,272985.0,-72971.0,147821.0,-1.80,-21.09,54.15,21.64,1.0,-0.49,2.00,0.00,X40,ATH,IT
40,INFY,2275.00,-20.32,41.0,H-LC,8.12,311001.0,-1887.0,173570.0,-1.66,-0.60,55.81,54.87,3.0,-0.01,2.28,6.01,X40,BTT,IT
42,ITC,452.00,-38.77,45.0,H-LC,2.15,197996.0,-2142.0,23027.0,1.20,-1.07,11.63,10.44,4.0,-0.09,1.45,4.43,X40,NTT,FMCG
83,VBL,671.64,-20.64,30.0,H-LC,8.19,285633.0,-30209.0,142874.0,-0.92,-9.56,50.02,35.67,5.0,-0.21,2.09,2.96,X40N,ATH,FMCG
1,ABB,7934.00,-40.84,42.0,H-LC,7.98,247296.0,-14323.0,133540.0,-0.94,-5.47,54.00,45.57,7.0,-0.11,1.81,5.23,AR,NTT,ELECTRICAL


In [22]:
# top 5 for average up
df_tmp = df_portfolio_features[(df_portfolio_features['Dev%_200'] > 5)].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
21,COFFEEDAY,80.00,-47.69,41.0,L-SC,41.27,77952.0,-35597.0,75645.0,-4.18,-31.35,97.04,35.27,268.0,-0.47,0.57,90.79,XR,NTT,HOTELS
61,SAMMAANCAP,326.00,-171.59,56.0,M-SC,5.87,82170.0,-20040.0,113427.0,0.40,-19.61,138.04,91.37,208.0,-0.18,0.60,33.39,XY25,NTT,FINANCE
49,MASFIN,398.61,-19.60,39.0,H-SC,12.50,91320.0,-6660.0,28264.0,-1.42,-6.80,30.95,22.05,152.0,-0.24,0.67,32.74,XR,ATH,FINANCE
50,NATIONALUM,244.55,-46.98,47.0,H-MC,10.89,106445.0,5941.0,23162.0,-2.32,5.91,21.76,28.96,79.0,0.26,0.78,41.89,MH,ATH,METALS
78,UJJIVANSFB,60.00,90.20,47.0,H-SC,10.14,120069.0,-22410.0,41928.0,-1.11,-15.73,34.92,13.70,163.0,-0.53,0.88,41.71,OX40N,NTT,BANKS


In [23]:
# top 5 RSP
df_tmp = df_portfolio_features.sort_values(by = 'RSP', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
64,SHALBY,327.00,1159.97,62.0,M-SC,12.64,171344.0,-9799.0,55258.0,-1.49,-5.41,32.25,25.10,235.0,-0.18,1.26,37.32,XY24,NTT,HEALTHCARE
61,SAMMAANCAP,326.00,-171.59,56.0,M-SC,5.87,82170.0,-20040.0,113427.0,0.40,-19.61,138.04,91.37,208.0,-0.18,0.60,33.39,XY25,NTT,FINANCE
71,TANLA,1963.11,-20.33,54.0,H-SC,16.94,213756.0,-57005.0,386962.0,-3.07,-21.05,181.03,121.86,133.0,-0.15,1.57,67.22,AR,ATH,IT
81,VAIBHAVGBL,521.00,52.07,49.0,H-SC,5.64,131767.0,-51008.0,162073.0,-3.07,-27.91,123.00,60.77,125.0,-0.31,0.97,18.02,XR,NTT,JEWELLERY
7,ATULAUTO,844.00,3548.65,48.0,M-SC,7.09,113740.0,-33628.0,85441.0,-1.98,-22.82,75.12,35.16,236.0,-0.39,0.83,16.17,XY24,NTT,AUTO


In [24]:
# Top N allocation
df_tmp = df_portfolio_features.sort_values(by = 'CurrAlloc%', ascending=False)
top_n_values = [10, 25, 50]

sum_df = pd.DataFrame({
    'Top_N': top_n_values,
    'Sum_Alloc%': [df_tmp['CurrAlloc%'].head(n).sum() for n in top_n_values]
})

sum_df

,Top_N,Sum_Alloc%
0,10,19.31
1,25,43.84
2,50,74.98


In [25]:
# market-cap-wise allocation
df_tmp = df_portfolio_features[cols]
df_tmp.groupby(df_tmp['Conviction'].str[-2:])['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
SC    43.85
LC    32.49
MC    23.68
Name: CurrAlloc%, dtype: float64

In [26]:
# criteria-wise allocation
df_portfolio_features.groupby('Criteria')['CurrAlloc%'].sum().sort_values(ascending=False)

Criteria
XY24     30.36
X40      14.78
X40N     11.38
XR       11.13
XY25     10.20
AR        8.30
OX40N     7.92
X200      1.68
MH        1.61
X5K       1.44
SR        1.22
Name: CurrAlloc%, dtype: float64

In [27]:
# conviction-wise allocation
df_portfolio_features.groupby('Conviction')['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
H-SC    29.36
H-LC    26.00
H-MC    20.74
M-SC    13.03
M-LC     5.43
M-MC     2.64
L-SC     1.46
L-LC     1.06
L-MC     0.30
Name: CurrAlloc%, dtype: float64

In [28]:
# sector-wise stats
df_tmp = df_portfolio_features.groupby('Category')[['CurrAlloc%', 'Current', 'Current P/L', 'FTT Amt']].sum().sort_values(by=['Current', 'Current P/L'], ascending=False)
df_tmp['Current P/L%'] = round(df_tmp['Current P/L'] * 100 / df_tmp['Current'], 2)
df_tmp['FTT%'] = round(df_tmp['FTT Amt'] * 100 / df_tmp['Current'], 2)
cols = ['CurrAlloc%', 'Current P/L%', 'FTT%']
df_tmp[cols].sort_values(by=['CurrAlloc%'], ascending=False)

,CurrAlloc%,Current P/L%,FTT%
Category,,,
FMCG,14.11,-6.64,41.83
IT,12.86,-19.95,85.35
FINANCE,10.05,-19.66,71.87
MISC,7.00,-20.44,87.39
BANKS,6.14,-19.08,81.72
PAINTS,5.70,-18.92,36.65
ELECTRICAL,5.47,-10.23,48.91
INSURANCE,3.79,-6.98,46.97
AC,3.57,0.75,15.87


In [29]:
# money to be made criteria-wise
df_portfolio_features.groupby('Criteria')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Criteria,,
XY24,3240060.0,22
XR,1393286.0,14
AR,1198488.0,9
X40,833355.0,10
X40N,651304.0,9
OX40N,566521.0,10
XY25,448154.0,7
SR,255281.0,2
X5K,121785.0,1


In [30]:
# money to be made conviction-wise
df_portfolio_features.groupby('Conviction')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Conviction,,
H-SC,3789807.0,29
M-SC,1466898.0,17
H-MC,1278441.0,15
H-LC,1272087.0,15
M-LC,409302.0,4
M-MC,317198.0,2
L-SC,262245.0,3
L-MC,61763.0,1
L-LC,40047.0,1


In [31]:
# money to be made criteria and conviction-wise
df_portfolio_features.groupby(['Conviction', 'Criteria'])['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

sum  count
Conviction Criteria                  
H-SC       XY24      1255861.0      6
M-SC       XY24       840355.0      7
H-SC       AR         815015.0      5
           XR         807450.0      7
H-MC       XY24       574511.0      4
H-LC       X40        528494.0      5
H-SC       X40N       320850.0      4
M-MC       XY24       317198.0      2
H-SC       OX40N      264706.0      4
           SR         255281.0      2
H-LC       X40N       242504.0      3
H-MC       X40        228359.0      4
H-LC       AR         212543.0      2
H-MC       XY25       179957.0      2
M-LC       XY24       174060.0      2
L-SC       XR         173485.0      2
M-SC       AR         170930.0      2
           OX40N      140212.0      4
           XR         125472.0      2
M-LC       X5K        121785.0      1
H-LC       XY25       114723.0      3
M-LC       XR         113457.0      1
M-SC       XY25       113427.0      1
H-MC       XR         111659.0      1
H-LC       X200        95748.0      1
L-SC       OX40N       88760.0      1
H-MC       X40N        87950.0      2
H-LC       XY24        78075.0      1
M-SC       X40         76502.0      1
H-MC       OX40N       72843.0      1
H-SC       MH          70644.0      1
L-MC       XR          61763.0      1
L-LC       XY25        40047.0      1
H-MC       MH          23162.0      1

In [32]:
# notebook execution time

end_time = time.time()
execution_time = round(end_time - start_time, 0)
print(f"Notebook execution time: {execution_time} seconds")

Notebook execution time: 26.0 seconds
